# Atlas Vector Search - Create Embeddings - Open Source - New Data

This notebook is a companion to the [Create Embeddings](https://www.mongodb.com/docs/atlas/atlas-vector-search/create-embeddings/) page. Refer to the page for set-up instructions and detailed explanations.

This notebook takes you through how to generate embeddings from **new data** by using the open-source ``nomic-embed-text-v1`` model.

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/create-embeddings/open-source-new-data.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
pip install --quiet --upgrade sentence-transformers pymongo einops

## Use an Embedding Model

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the embedding model
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

# Define a function to generate embeddings
def get_embedding(data, precision="float32"):
   return model.encode(data, precision=precision).tolist()

# Generate an embedding
embedding = get_embedding("foo")
print(embedding)

### (Optional) Compress your embeddings

Optionally, run the following code to define a function that converts your embeddings into BSON `binData` vectors for [efficient storage and retrieval](https://www.mongodb.com/docs/atlas/atlas-vector-search/create-embeddings/#vector-compression).

In [ ]:
from bson.binary import Binary 
from bson.binary import BinaryVectorDtype

# Define a function to generate BSON vectors
def generate_bson_vector(vector, vector_dtype):
   return Binary.from_vector(vector, vector_dtype)

# Generate BSON vector from the sample float32 embedding
bson_float32_embedding = generate_bson_vector(embedding, BinaryVectorDtype.FLOAT32)

# Print the converted embedding
print(f"The converted BSON embedding is: {bson_float32_embedding}")

## Generate Embeddings

In [ ]:
# Sample data
texts = [
  "Titanic: The story of the 1912 sinking of the largest luxury liner ever built",
  "The Lion King: Lion cub and future king Simba searches for his identity",
  "Avatar: A marine is dispatched to the moon Pandora on a unique mission"
]

In [ ]:
# Generate embeddings from the sample data
embeddings = []
for text in texts:
 embedding = get_embedding(text)

 # Uncomment the following line to convert to BSON vectors
 # embedding = generate_bson_vector(embedding, BinaryVectorDtype.FLOAT32)
 
 embeddings.append(embedding)

 # Print the embeddings
 print(f"\nText: {text}")
 print(f"Embedding: {embedding[:3]}... (truncated)")

## Ingest Embeddings into Atlas

In [ ]:
def create_docs_with_embeddings(embeddings, data):
   docs = []
   for i, (embedding, text) in enumerate(zip(embeddings, data)):
      doc = {
            "_id": i,
            "text": text,
            "embedding": embedding,
      }
      docs.append(doc)
   return docs

In [ ]:
# Create documents with embeddings and sample data
docs = create_docs_with_embeddings(embeddings, texts)

In [ ]:
import pymongo

# Connect to your Atlas cluster
mongo_client = pymongo.MongoClient("<connection-string>")
db = mongo_client["sample_db"]
collection = db["embeddings"]

# Ingest data into Atlas
collection.insert_many(docs)

## Index and Query Your Embeddings

In [ ]:
from pymongo.operations import SearchIndexModel
import time

# Create your index model, then create the search index
search_index_model = SearchIndexModel(
  definition = {
    "fields": [
      {
        "type": "vector",
        "path": "embedding",
        "similarity": "dotProduct",
        "numDimensions": 768
      }
    ]
  },
  name="vector_index",
  type="vectorSearch"
)
result = collection.create_search_index(model=search_index_model)

# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None
if predicate is None:
  predicate = lambda index: index.get("queryable") is True

while True:
  indices = list(collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):
    break
  time.sleep(5)
print(result + " is ready for querying.")

In [ ]:
# Generate embedding for the search query
query_embedding = get_embedding("ocean tragedy")

# Sample vector search pipeline
pipeline = [
   {
      "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "exact": True,
            "limit": 5
      }
   }, 
   {
      "$project": {
         "_id": 0, 
         "text": 1,
         "score": {
            "$meta": "vectorSearchScore"
         }
      }
   }
]

# Execute the search
results = collection.aggregate(pipeline)

# Print results
for i in results:
   print(i)
